# Loading Models

This notebook serves to download model checkpoint files hosted from the [Microsoft Aurora Hugging Face repository](https://huggingface.co/microsoft/aurora) and register these as Azure Machine Learning model assets.

This notebook should be run in the Notebooks area of Azure Machine Learning. It does not require GPU capable compute.

In [ ]:
%pip install huggingface-hub

In [ ]:
import sys
from pathlib import Path

from azure.ai.ml.entities import Model
from azure.core.exceptions import ResourceNotFoundError
from azure.identity import DefaultAzureCredential
from huggingface_hub import hf_hub_download

# insert parent directory to path for proper absolute local imports
sys.path.insert(0, str(Path.cwd().parent.parent.resolve()))
from setup.common.utils import create_mlclient, get_latest_asset


Define the Microsoft Aurora Hugging Face repository and model checkpoint file to download.

In [ ]:
HF_REPOSITORY = "microsoft/aurora"
MODEL_FILENAME = "aurora-0.25-pretrained.ckpt"

Obtain and create necessary environment parameters and Azure interface objects.

In [ ]:
az_cred = DefaultAzureCredential()
ml_client = create_mlclient(local=False)

Download the model checkpoint file to the local `huggingface-hub` cache directory on the compute instance.

In [ ]:
path = hf_hub_download(repo_id=HF_REPOSITORY, filename=MODEL_FILENAME)
print(f"Model downloaded to local cache: {path}")

Register the model as an AML model asset.

In [ ]:
# names can only contain alphanumeric chars and dashes, so replace the decimal with "p"
model_name = MODEL_FILENAME.removesuffix(".ckpt").replace(".", "p")
try:
    new = int(get_latest_asset(ml_client.models, model_name).version) + 1
except ResourceNotFoundError:
    new = 1

model_asset = Model(name=model_name, version=str(new), path=path)
ml_client.models.create_or_update(model_asset)
print(
    f"Created or updated asset: name={model_asset.name}, "
    f"version={model_asset.version}, path={model_asset.path}",
)